In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import random

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# 自定义 MAPE 损失函数
class MAPE_Loss(nn.Module):
    def __init__(self):
        super(MAPE_Loss, self).__init__()

    def forward(self, y_pred, y_true):
        epsilon = 1e-8  # 避免除零
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# 自定义 RMSE 损失函数
class RMSE_Loss(nn.Module):
    def __init__(self):
        super(RMSE_Loss, self).__init__()

    def forward(self, y_pred, y_true):
        return torch.sqrt(torch.mean((y_true - y_pred) ** 2))

In [3]:
# 读取数据
data = pd.read_csv("../data/dataset.csv")

# 数据分割
data['target_class'] = pd.qcut(data['Cs'], q=10, labels=False)
X = data.drop(['Cs', 'target_class'], axis=1).values
y = data['Cs'].values
stratify_column = data['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

# 数据标准化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 将数据转换为 PyTorch 张量并移动到 GPU
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# 创建 DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# 定义 ANN 模型（确保在使用前定义）
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 12)
        self.layer2 = nn.Linear(12, 90)
        self.layer3 = nn.Linear(90, 90)
        self.layer4 = nn.Linear(90, 60)
        self.layer5 = nn.Linear(60, 70)
        self.layer6 = nn.Linear(70, 30)
        self.output = nn.Linear(30, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.relu(self.layer4(x))
        x = torch.relu(self.layer5(x))
        x = torch.relu(self.layer6(x))
        x = self.output(x)
        return x

# 初始化模型并将其移到 GPU（如果可用）
input_dim = X_train.shape[1]
model = ANN(input_dim).to(device)

# 定义损失函数和优化器
loss_function = MAPE_Loss().to(device)  # 自定义损失函数也要移到 GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
from function import metrics_to_dataframe

# 重新定义模型结构
model = ANN(input_dim).to(device)  # 使用相同的模型结构

# 加载模型参数，确保使用 weights_only=True 来提高安全性
model.load_state_dict(torch.load("ann_model.pth", weights_only=True))  # 加载模型参数

# 评估模型 (确保在 GPU 上)
model.eval()
with torch.no_grad():
    y_train_pred = model(X_train_tensor).cpu().numpy()  # 转换为 CPU 数据，便于后续处理
    y_test_pred = model(X_test_tensor).cpu().numpy()

# 绘制实际 vs 预测图
# plot_actual_vs_predicted(y_train, y_train_pred, y_test, y_test_pred, 'Artificial Neural Network', 'ann.png')

# 计算并显示评估指标
ann_metrics = metrics_to_dataframe(y_train, y_train_pred, y_test, y_test_pred, 'ANN')
ann_metrics

,model,R2_train,MAE_train,MAPE_train,RMSE_train,R2_test,MAE_test,MAPE_test,RMSE_test
0,ANN,0.943932,5.739903,6.449726,9.452667,0.963017,5.347572,6.180497,7.632014


In [7]:
# 打印模型的详细信息
print("Model Architecture:")
print(model)  # 打印模型的结构

# 打印模型中每一层的参数
print("\nModel Parameters:")
for name, param in model.named_parameters():
    print(f"{name}: {param.size()}")  # 输出参数的名称及其形状

Model Architecture:
ANN(
  (layer1): Linear(in_features=12, out_features=12, bias=True)
  (layer2): Linear(in_features=12, out_features=90, bias=True)
  (layer3): Linear(in_features=90, out_features=90, bias=True)
  (layer4): Linear(in_features=90, out_features=60, bias=True)
  (layer5): Linear(in_features=60, out_features=70, bias=True)
  (layer6): Linear(in_features=70, out_features=30, bias=True)
  (output): Linear(in_features=30, out_features=1, bias=True)
)

Model Parameters:
layer1.weight: torch.Size([12, 12])
layer1.bias: torch.Size([12])
layer2.weight: torch.Size([90, 12])
layer2.bias: torch.Size([90])
layer3.weight: torch.Size([90, 90])
layer3.bias: torch.Size([90])
layer4.weight: torch.Size([60, 90])
layer4.bias: torch.Size([60])
layer5.weight: torch.Size([70, 60])
layer5.bias: torch.Size([70])
layer6.weight: torch.Size([30, 70])
layer6.bias: torch.Size([30])
output.weight: torch.Size([1, 30])
output.bias: torch.Size([1])
